In [ ]:
# !pip install smolagents sqlalchemy plotly

from smolagents import CodeAgent, InferenceClientModel, tool
from dotenv import load_dotenv
load_dotenv()

import datetime
import pandas as pd
import json
from datetime import datetime
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    DateTime,
    ForeignKey,
    insert,
    inspect,
    text,
)
from sqlalchemy.sql import func

In [ ]:

# Database setup (keeping your existing schema and data)
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        action = insert(table).values(**row)
        with engine.begin() as actor:
            actor.execute(action)


In [ ]:

# Table definitions (keeping your existing schema)
business_category = Table(
    "business_category",
    metadata_obj,
    Column("category_id", Integer, primary_key=True),
    Column("category_name", String(64)),
)

customers = Table(
    "customers",
    metadata_obj,
    Column("customer_id", Integer, primary_key=True),
    Column("customer_name", String(64)),
    Column("category_id", Integer),
)

departments = Table(
    "departments",
    metadata_obj,
    Column("department_id", Integer, primary_key=True),
    Column("department_name", String(64)),
    Column("department_size", Integer)
)

job_desk = Table(
    "job_desk",
    metadata_obj,
    Column("job_desk_id", Integer, primary_key=True),
    Column("job_desk_name", String(64))
)

employees = Table(
    "employees",
    metadata_obj,
    Column("employee_id", Integer, primary_key=True),
    Column("employee_name", String(64), nullable=False),
    Column("job_desk_id", Integer),
    Column("department_id", Integer)
)

products = Table(
    "products",
    metadata_obj,
    Column("product_id", Integer, primary_key=True),
    Column("product_name", String(128), nullable=False),
    Column("price", Float, nullable=False),
    Column("stock", Integer, nullable=False, default=0)
)

transaction_types = Table(
    "transaction_types",
    metadata_obj,
    Column("transaction_type_id", Integer, primary_key=True),
    Column("transaction_type_name", String(64))
)

transactions = Table(
    "transactions",
    metadata_obj,
    Column("transaction_id", Integer, primary_key=True),
    Column("customer_id", Integer),
    Column("employee_id", Integer),
    Column("approver_id", Integer),
    Column("product_id", Integer),
    Column("transaction_type_id", Integer),
    Column("total_amount", Float),
    Column("remaining_amount", Float),
    Column("created_at", DateTime(timezone=True), server_default=func.now()),
    Column("due_at", DateTime(timezone=True)),
)

metadata_obj.create_all(engine)


In [ ]:

# Sample data (keeping your existing data)
business_category_samples = [
    {"category_id": 1, "category_name": "Retail"},
    {"category_id": 2, "category_name": "Wholesale"},
    {"category_id": 3, "category_name": "Corporate"},
    {"category_id": 4, "category_name": "Government"},
    {"category_id": 5, "category_name": "Individual"},
]

customers_samples = [
    {"customer_id": 1, "customer_name": "Alpha Mart", "category_id": 1},
    {"customer_id": 2, "customer_name": "Beta Supplies", "category_id": 2},
    {"customer_id": 3, "customer_name": "Gamma Corp", "category_id": 3},
    {"customer_id": 4, "customer_name": "City Council", "category_id": 4},
    {"customer_id": 5, "customer_name": "John Doe", "category_id": 5},
]

departments_samples = [
    {"department_id": 1, "department_name": "Sales", "department_size": 15},
    {"department_id": 2, "department_name": "Finance", "department_size": 10},
    {"department_id": 3, "department_name": "HR", "department_size": 5},
    {"department_id": 4, "department_name": "IT", "department_size": 8},
    {"department_id": 5, "department_name": "Logistics", "department_size": 12},
]

job_desk_samples = [
    {"job_desk_id": 1, "job_desk_name": "Sales Representative"},
    {"job_desk_id": 2, "job_desk_name": "Accountant"},
    {"job_desk_id": 3, "job_desk_name": "HR Specialist"},
    {"job_desk_id": 4, "job_desk_name": "Software Engineer"},
    {"job_desk_id": 5, "job_desk_name": "Warehouse Manager"},
]

employees_samples = [
    {"employee_id": 1, "employee_name": "Alice Johnson", "job_desk_id": 1, "department_id": 1},
    {"employee_id": 2, "employee_name": "Bob Smith", "job_desk_id": 2, "department_id": 2},
    {"employee_id": 3, "employee_name": "Charlie Brown", "job_desk_id": 3, "department_id": 3},
    {"employee_id": 4, "employee_name": "Diana Prince", "job_desk_id": 4, "department_id": 4},
    {"employee_id": 5, "employee_name": "Ethan Hunt", "job_desk_id": 5, "department_id": 5},
]

products_samples = [
    {"product_id": 1, "product_name": "Laptop Pro", "price": 1200.00, "stock": 25},
    {"product_id": 2, "product_name": "Office Chair", "price": 150.00, "stock": 100},
    {"product_id": 3, "product_name": "Printer X200", "price": 300.00, "stock": 40},
    {"product_id": 4, "product_name": "Desk Set", "price": 250.00, "stock": 60},
    {"product_id": 5, "product_name": "Monitor HD", "price": 200.00, "stock": 75},
]

transaction_types_samples = [
    {"transaction_type_id": 1, "transaction_type_name": "purchase"},
    {"transaction_type_id": 2, "transaction_type_name": "refund"},
    {"transaction_type_id": 3, "transaction_type_name": "credit"},
    {"transaction_type_id": 4, "transaction_type_name": "installment"},
    {"transaction_type_id": 5, "transaction_type_name": "service"},
]
transactions_samples = [
    {
        "transaction_id": 1, "customer_id": 1, "employee_id": 1, "approver_id": 2,
        "product_id": 1, "transaction_type_id": 1, "total_amount": 1200.00,
        "remaining_amount": 0.00,
        "created_at": datetime(2025, 8, 1, 10, 0, 0),
        "due_at": None
    },
    {
        "transaction_id": 2, "customer_id": 2, "employee_id": 1, "approver_id": 3,
        "product_id": 2, "transaction_type_id": 4, "total_amount": 1500.00,
        "remaining_amount": 500.00,
        "created_at": datetime(2025, 8, 2, 11, 30, 0),
        "due_at": datetime(2025, 9, 2, 11, 30, 0)  # Not overdue yet
    },
    {
        "transaction_id": 3, "customer_id": 3, "employee_id": 2, "approver_id": 4,
        "product_id": 3, "transaction_type_id": 1, "total_amount": 900.00,
        "remaining_amount": 0.00,
        "created_at": datetime(2025, 8, 3, 14, 20, 0),
        "due_at": None
    },
    {
        "transaction_id": 4, "customer_id": 4, "employee_id": 3, "approver_id": 5,
        "product_id": 4, "transaction_type_id": 3, "total_amount": 250.00,
        "remaining_amount": 0.00,
        "created_at": datetime(2025, 8, 4, 9, 15, 0),
        "due_at": None
    },
    {
        "transaction_id": 5, "customer_id": 5, "employee_id": 4, "approver_id": 1,
        "product_id": 5, "transaction_type_id": 2, "total_amount": -200.00,
        "remaining_amount": 0.00,
        "created_at": datetime(2025, 8, 5, 16, 45, 0),
        "due_at": None
    },
    # Overdue installment
    {
        "transaction_id": 6, "customer_id": 1, "employee_id": 2, "approver_id": 3,
        "product_id": 3, "transaction_type_id": 4, "total_amount": 600.00,
        "remaining_amount": 300.00,
        "created_at": datetime(2025, 7, 15, 10, 0, 0),
        "due_at": datetime(2025, 8, 15, 10, 0, 0)  # Overdue by 10 days
    },
    # Overdue installment (long overdue)
    {
        "transaction_id": 7, "customer_id": 3, "employee_id": 1, "approver_id": 2,
        "product_id": 1, "transaction_type_id": 4, "total_amount": 1200.00,
        "remaining_amount": 800.00,
        "created_at": datetime(2025, 6, 1, 14, 0, 0),
        "due_at": datetime(2025, 7, 1, 14, 0, 0)  # Overdue by almost 2 months
    },
    # On-time installment (still active)
    {
        "transaction_id": 8, "customer_id": 5, "employee_id": 4, "approver_id": 1,
        "product_id": 2, "transaction_type_id": 4, "total_amount": 450.00,
        "remaining_amount": 150.00,
        "created_at": datetime(2025, 8, 10, 12, 0, 0),
        "due_at": datetime(2025, 9, 10, 12, 0, 0)  # Not overdue yet
    },
]


In [ ]:

# Insert all data
sample_data = {
    business_category: business_category_samples,
    customers: customers_samples,
    departments: departments_samples,
    job_desk: job_desk_samples,
    employees: employees_samples,
    products: products_samples,
    transaction_types: transaction_types_samples,
    transactions: transactions_samples,
}

for table, rows in sample_data.items():
    insert_rows_into_table(rows, table)

In [ ]:
def display_transactions_table_sqlalchemy():
    """
    Queries and displays the 'transactions' table using SQLAlchemy Table object.
    """
    try:
        import pandas as pd
        from sqlalchemy import select

        # Use the defined SQLAlchemy Table object
        query = select(customers)
        with engine.connect() as conn:
            result = conn.execute(query)
            rows = result.fetchall()
            if not rows:
                print("The 'transactions' table is empty.")
            else:
                print("Transactions Table (SQLAlchemy):")
                # Convert to DataFrame for better display
                df = pd.DataFrame(rows, columns=result.keys())
                display(df)
    except Exception as e:
        print(f"Error displaying transactions table (SQLAlchemy): {e}")

# Call the function to display the transactions table using SQLAlchemy
display_transactions_table_sqlalchemy()

Transactions Table (SQLAlchemy):


,customer_id,customer_name,category_id
0,1,Alpha Mart,1
1,2,Beta Supplies,2
2,3,Gamma Corp,3
3,4,City Council,4
4,5,John Doe,5


In [ ]:

# Schema validation function
def validate_sql_query(query: str) -> tuple[bool, str]:
    """
    Validates SQL query against known schema to prevent hallucinations.
    Returns (is_valid, error_message)
    """
    # Valid table names (exact match required)
    valid_tables = {
        'transactions', 'transaction_types', 'products', 'customers',
        'employees', 'departments', 'business_category', 'job_desk'
    }

    # Valid column names by table
    valid_columns = {
        'transactions': {'transaction_id', 'customer_id', 'employee_id', 'approver_id',
                        'product_id', 'transaction_type_id', 'total_amount', 'remaining_amount',
                        'created_at', 'due_at'},
        'transaction_types': {'transaction_type_id', 'transaction_type_name'},
        'products': {'product_id', 'product_name', 'price', 'stock'},
        'customers': {'customer_id', 'customer_name', 'category_id'},
        'employees': {'employee_id', 'employee_name', 'job_desk_id', 'department_id'},
        'departments': {'department_id', 'department_name', 'department_size'},
        'business_category': {'category_id', 'category_name'},
        'job_desk': {'job_desk_id', 'job_desk_name'}
    }

    query_lower = query.lower()

    # Check for invalid table references
    import re

    # Find table references after FROM and JOIN
    table_patterns = [
        r'from\s+(\w+)',
        r'join\s+(\w+)',
        r'left\s+join\s+(\w+)',
        r'right\s+join\s+(\w+)',
        r'inner\s+join\s+(\w+)',
        r'outer\s+join\s+(\w+)'

    ]

    referenced_tables = set()
    for pattern in table_patterns:
        matches = re.findall(pattern, query_lower)
        referenced_tables.update(matches)

    # Check for invalid tables
    invalid_tables = referenced_tables - valid_tables
    if invalid_tables:
        return False, f"Invalid table(s) found: {invalid_tables}. Valid tables: {valid_tables}"

    # Check for common hallucinated table names
    hallucinated_tables = {
        'payments', 'installments', 'salesreps', 'sales_reps', 'orders',
        'invoices', 'customers_payments', 'payment_status'
    }

    found_hallucinated = referenced_tables & hallucinated_tables
    if found_hallucinated:
        return False, f"Hallucinated table(s) detected: {found_hallucinated}. Use only: {valid_tables}"

    # Check for common hallucinated column patterns
    hallucinated_patterns = [
        r'customerid', r'customername', r'salesrepid', r'salesrepname',
        r'amountowed', r'paymentstatus', r'duedate', r'amountpaid',
        r'amountdue', r'installmentid'
    ]

    for pattern in hallucinated_patterns:
        if re.search(pattern, query_lower):
            return False, f"Possible hallucinated column detected. Check column names against schema."

    return True, "Query appears valid"


In [ ]:
from smolagents import tool
# Improved SQL Engine Tool with validation
@tool
def sql_engine(query: str) -> str:
    """
    Executes SQL queries on the database with schema validation.
    Returns a string representation of the result.

    Here are the tables' descriptions:

    transactions:
      - transaction_id (INTEGER)          -- unique transaction record
      - customer_id (INTEGER)             -- links to customers
      - employee_id (INTEGER)             -- links to employees (who sold it)
      - approver_id (INTEGER)             -- approving manager, if applicable
      - product_id (INTEGER)              -- links to products
      - transaction_type_id (INTEGER)     -- links to transaction_types
      - total_amount (FLOAT)              -- full value of the transaction
      - remaining_amount (FLOAT)          -- unpaid balance. If > 0, customer still owes money
      - created_at (DATETIME)             -- when transaction was created
      - due_at (DATETIME)                 -- payment due date

    customers:
      - customer_id (INTEGER)
      - customer_name (VARCHAR)
      - category_id (INTEGER)

    employees:
      - employee_id (INTEGER)
      - employee_name (VARCHAR)
      - job_desk_id (INTEGER)
      - department_id (INTEGER)

    transaction_types:
      - transaction_type_id (INTEGER)
      - transaction_type_name (VARCHAR)   -- fixed values: 'purchase', 'refund', 'credit', 'installments', 'service'

    products:
      - product_id (INTEGER)
      - product_name (VARCHAR)
      - price (FLOAT)
      - stock (INTEGER)

    departments:
      - department_id (INTEGER)
      - department_name (VARCHAR)
      - department_size (INTEGER)

    business_category:
      - category_id (INTEGER)
      - category_name (VARCHAR)

    job_desk:
      - job_desk_id (INTEGER)
      - job_desk_name (VARCHAR)

    DEFINITIONS / BUSINESS LOGIC:
    - **Overdue payment**: A transaction is overdue if:
      1. `remaining_amount > 0` (customer still owes money)
      2. `due_at < CURRENT_DATE` (the due date has already passed)
    - **Relevant transaction types**: Only `credit` and `installments` transactions can become overdue.
      Purchases are usually fully paid, refunds reduce balances, and services may not have installments.
    - **Amount still owed**: Taken from `transactions.remaining_amount`.
    - **Who sold it**: The `employee_id` in `transactions` joins to `employees.employee_name`.
    - **Days overdue**: Use `julianday('now') - julianday(transactions.due_at)` in SQLite.

    IMPORTANT:
    - There are NO tables named 'payments', 'installments', 'credit_transactions', or 'salesreps'.
    - Use `customers.customer_name`, not `CustomerName`.
    - Use `employees.employee_name`, not `SalesRepName`.
    - Use SQLite date functions (`julianday`, `datetime`, etc.).

    Args:
        query: The query to perform. This should be correct SQL.

    """

    try:
        with engine.connect() as conn:
            result = conn.execute(text(query))
            rows = result.fetchall()

            if not rows:
                return "✅ Query executed successfully but returned no results."

            columns = result.keys()
            output = "✅ Query executed successfully!\n\nResults:\n"
            output += " | ".join(str(col) for col in columns) + "\n"
            output += "-" * (len(" | ".join(str(col) for col in columns))) + "\n"

            for row in rows:
                output += " | ".join(str(val) for val in row) + "\n"

            return output

    except Exception as e:
        return f"❌ SQL EXECUTION ERROR: {str(e)}\n\nCheck your SQL syntax and ensure you're using the correct table/column names from the schema."

# Validate query first
    # is_valid, error_msg = validate_sql_query(query)
    # if not is_valid:
    #     return f"❌ SCHEMA VALIDATION ERROR: {error_msg}\n\nPlease rewrite using only the exact table and column names from the schema above."

# # Enhanced SQL to DataFrame Tool for the analyst
@tool
def sql_to_dataframe(query: str) -> str:
    """
    Executes SQL query and converts results to pandas DataFrame for analysis.
    Returns JSON string containing DataFrame info and sample data.

    Args:
        query: The query to perform. This should be correct SQL.

    """
    try:
        import pandas as pd
        import json

        # Execute query and get DataFrame
        df = pd.read_sql_query(query, engine)

        if df.empty:
            return json.dumps({
                "status": "empty",
                "message": "Query returned no results",
                "columns": [],
                "sample_data": []
            })

        # Return structured info about the DataFrame
        result = {
            "status": "success",
            "shape": df.shape,
            "columns": df.columns.tolist(),
            "dtypes": df.dtypes.astype(str).to_dict(),
            "sample_data": df.head(10).to_dict('records'),
            "summary_stats": df.describe().to_dict() if df.select_dtypes(include=['number']).shape[1] > 0 else {}
        }

        return json.dumps(result, default=str, indent=2)

    except Exception as e:
        return json.dumps({
            "status": "error",
            "error": str(e)
        })


In [ ]:

# Business Intelligence Agent (replaces planning agent with better focus)
bi_agent = CodeAgent(
    name="business_intelligence_agent",
    model=InferenceClientModel(),
    description=(
        "Business Intelligence Agent that translates business questions into SQL queries. "
        "This agent understands business terminology, maps it to database schema, "
        "and generates appropriate SQL queries with business context."
    ),
    tools=[sql_engine],
    additional_authorized_imports=["datetime", "json", "sqlalchemy"],
)
bi_prompt = """
You are a Business Intelligence SQL Agent. Your role is to convert natural language business questions into accurate, efficient SQL using only the provided schema.

Explicit rules you must follow:

Use only given tables/columns.
Return SQL in a Python code block with triple quotes.
Keep queries clean, well-aliased, and optimized.
If impossible with schema, reply: "Cannot generate query: [reason]."
No explanations unless asked.

Example:

Schema:
orders(order_id, customer_id, order_date, total_amount)
customers(customer_id, name, region)

Q: "Which region had the highest sales last quarter?"

A:
query = '''
SELECT c.region, SUM(o.total_amount) AS total_sales
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.order_date BETWEEN DATE '2024-04-01' AND DATE '2024-06-30'
GROUP BY c.region
ORDER BY total_sales DESC
LIMIT 1;
'''

"""

bi_agent.prompt_templates['system_prompt'] += bi_prompt

In [ ]:

# Data Analysis Agent (improved analyst)
analysis_agent = CodeAgent(
    name="data_analysis_agent",
    model=InferenceClientModel(),
    description=(
        "Data Analysis Agent that processes SQL results, creates DataFrames, "
        "performs analysis, and generates visualizations and insights."
    ),
    tools=[],
    additional_authorized_imports=[
        "pandas", "pandas.*", "numpy","numpy.*", "json", "plotly", "plotly.express",
        "plotly.graph_objects", "datetime", "statistics"
    ],
)

analysis_agent_prompt = """
You are the Data Analysis Agent.
Your Role:
1. Execute Analysis: Take SQL queries and convert results to pandas DataFrames
2. Data Processing: Clean, validate, and enrich the data
3. Generate Insights: Identify patterns, trends, and key findings
4. Create Visualizations: Generate appropriate charts using Plotly
5. Business Reporting: Provide executive-ready summaries
Your Process:
1. Data Acquisition: Use sql_to_dataframe tool to get structured data
2. Data Validation: Check for completeness, accuracy, and consistency
3. Analysis: Calculate key metrics, identify trends, find outliers
4. Visualization: Create charts that best represent the insights
5. Reporting: Summarize findings in business-friendly language
Visualization Guidelines:
- Comparisons: Bar charts for categorical comparisons
- Trends: Line charts for time-series data
- Distributions: Histograms for value distributions
- Relationships: Scatter plots for correlations
- Proportions: Pie charts for part-to-whole relationships
Analysis Types:
- Descriptive: What happened? (totals, averages, distributions)
- Diagnostic: Why did it happen? (comparisons, correlations)
- Predictive: What might happen? (trends, patterns)
- Prescriptive: What should we do? (recommendations)
Output Format:
Provide a structured analysis with:
1. Data Summary: Shape, quality, key statistics
2. Key Findings: Top 3-5 business insights
3. Visualizations: Plotly chart specifications in JSON
4. Recommendations: Actionable business advice
5. Technical Details: Any data quality issues or limitations
Always focus on actionable business insights and present data in a clear, professional manner.
"""

analysis_agent.prompt_templates['system_prompt'] += analysis_agent_prompt

In [ ]:

# Orchestrator Agent (improved manager)
orchestrator_agent = CodeAgent(
    name="orchestrator_agent",
    model=InferenceClientModel(),
    description=(
        "Orchestrator that manages the BI and Analysis agents. "
        "Routes business questions through the proper workflow: "
        "Business question -> SQL generation -> Data analysis -> Insights & recommendations"
    ),
    tools=[],
    managed_agents=[bi_agent, analysis_agent],
    additional_authorized_imports=["json", "time"]
)

orchestrator_prompt = """
You are the Orchestrator Agent for the Business Intelligence system.
Your Role:
Manage the workflow between Business Intelligence Agent and Data Analysis Agent to provide complete business insights.

Workflow:
1. Route to BI Agent: Send business questions to generate appropriate SQL queries and get initial results
2. Route to Analysis Agent: Send the SQL query to get structured analysis, insights, and visualizations
3. Synthesize Results: Combine both outputs into comprehensive business report

👥 Your Team:
- BI Agent: Translates business questions → SQL queries → Raw results
- Analysis Agent: SQL queries → DataFrames → Analysis → Visualizations → Insights

📋 Your Process:
1. Understand Request: Parse the user's business question
2. Delegate to BI: Get SQL query and initial results
3. Delegate to Analysis: Get structured analysis and visualizations
4. Compile Report: Create executive summary combining both outputs
5. Quality Check: Ensure results answer the original question

Output Format:
Create a comprehensive business report with:

Executive Summary: Key findings in 2-3 sentences
Detailed Results: What the data shows
Analysis & Insights: Why it matters for the business
Visualizations: Charts and graphs (from Analysis Agent)
Recommendations: Specific actions to take
Technical Notes: Any limitations or considerations

Example Flow:
User: "Which customers have overdue payments?"

Step 1: Ask BI Agent to translate question and get data
Step 2: Ask Analysis Agent to analyze the results
Step 3: Compile comprehensive report

Always ensure the final output directly answers the user's original business question with actionable insights.
"""

orchestrator_agent.prompt_templates['system_prompt'] += orchestrator_prompt

In [ ]:

# Test the improved system
print("🚀 Improved TTSQL Multi-Agent System Ready!")
print("\n" + "="*60)
print("SYSTEM ARCHITECTURE:")
print("="*60)
print("1. 📊 Business Intelligence Agent - Translates questions → SQL")
print("2. 🔍 Data Analysis Agent - SQL → DataFrames → Insights")
print("3. 🎯 Orchestrator Agent - Manages workflow & compiles reports")
print("="*60)

# Test query
user_query = input("Enter query here: ")

print(f"\n🔍 Test Query: {user_query}")
print("\n" + "-"*60)

# Run the orchestrator
result = orchestrator_agent.run(user_query)

🚀 Improved TTSQL Multi-Agent System Ready!

SYSTEM ARCHITECTURE:
1. 📊 Business Intelligence Agent - Translates questions → SQL
2. 🔍 Data Analysis Agent - SQL → DataFrames → Insights
3. 🎯 Orchestrator Agent - Manages workflow & compiles reports
Enter query here: Top 10 customers with the highest debt

🔍 Test Query: Top 10 customers with the highest debt

------------------------------------------------------------


╭───────────────────────────────────────── New run - orchestrator_agent ──────────────────────────────────────────╮
│                                                                                                                 │
│ Top 10 customers with the highest debt                                                                          │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_customers_query = business_intelligence_agent(                                                               
      task="Generate an SQL query to find the top 10 customers with the highest debt.",                            
      additional_args={}                                                                                           
  )                                                                                                                
  print(top_customers_query)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────── New run - business_intelligence_agent ─────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'business_intelligence_agent'.                                                     │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Generate an SQL query to find the top 10 customers with the highest debt.                                       │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

...

[Step 2: Duration 1.67 seconds| Input tokens: 5,432 | Output tokens: 189]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: 
Root=1-68ad29b7-58816bc750724da4514e9b90;ee2e8765-8693-4c9a-9150-ab762134a90b)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 3: Duration 0.03 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-68ad29b7-58816bc750724da4514e9b90;ee2e8765-8693-4c9a-9150-ab762134a90b)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.